#  T M V A Multiclass Application
This macro provides a simple example on how to use the trained multiclass
classifiers within an analysis module
- Project   : TMVA - a Root-integrated toolkit for multivariate data analysis
- Package   : TMVA
- Root Macro: TMVAMulticlassApplication



**Author:** Andreas Hoecker  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:50 AM.</small></i>

In [1]:
%%cpp -d
#include <cstdlib>
#include <iostream>
#include <map>
#include <string>
#include <vector>

#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TSystem.h"
#include "TROOT.h"
#include "TStopwatch.h"
#include "TH1F.h"

#include "TMVA/Tools.h"
#include "TMVA/Reader.h"

using namespace TMVA;

 Arguments are defined. 

In [2]:
TString myMethodList = "";

In [3]:
TMVA::Tools::Instance();

---------------------------------------------------------------
 Default MVA methods to be trained + tested

In [4]:
std::map<std::string,int> Use;
Use["MLP"]             = 1;
Use["BDTG"]            = 1;
Use["DNN_CPU"] = 0;
Use["FDA_GA"]          = 0;
Use["PDEFoam"]         = 0;

---------------------------------------------------------------

In [5]:
std::cout << std::endl;
std::cout << "==> Start TMVAMulticlassApp" << std::endl;
if (myMethodList != "") {
   for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++) it->second = 0;

   std::vector<TString> mlist = gTools().SplitString( myMethodList, ',' );
   for (UInt_t i=0; i<mlist.size(); i++) {
      std::string regMethod(mlist[i]);

      if (Use.find(regMethod) == Use.end()) {
         std::cout << "Method \"" << regMethod << "\" not known in TMVA under this name. Choose among the following:" << std::endl;
         for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++) std::cout << it->first << " " << std::endl;
         std::cout << std::endl;
         return;
      }
      Use[regMethod] = 1;
   }
}


==> Start TMVAMulticlassApp


Create the reader object

In [6]:
TMVA::Reader *reader = new TMVA::Reader( "!Color:!Silent" );

Create a set of variables and declare them to the reader
 - the variable names must corresponds in name and type to
 those given in the weight file(s) that you use

In [7]:
Float_t var1, var2, var3, var4;
reader->AddVariable( "var1", &var1 );
reader->AddVariable( "var2", &var2 );
reader->AddVariable( "var3", &var3 );
reader->AddVariable( "var4", &var4 );

Book the mva methods

In [8]:
TString dir    = "dataset/weights/";
TString prefix = "TMVAMulticlass";

for (std::map<std::string,int>::iterator it = Use.begin(); it != Use.end(); it++) {
   if (it->second) {
     TString methodName = TString(it->first) + TString(" method");
     TString weightfile = dir + prefix + TString("_") + TString(it->first) + TString(".weights.xml");
     reader->BookMVA( methodName, weightfile );
   }
}

                         : Booking "BDTG method" of type "BDT" from dataset/weights/TMVAMulticlass_BDTG.weights.xml.
                         : Reading weight file: dataset/weights/TMVAMulticlass_BDTG.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "bg0"
<HEADER> DataSetInfo              : [Default] : Added class "bg1"
<HEADER> DataSetInfo              : [Default] : Added class "bg2"
                         : Booked classifier "BDTG" of type: "BDT"
                         : Booking "MLP method" of type "MLP" from dataset/weights/TMVAMulticlass_MLP.weights.xml.
                         : Reading weight file: dataset/weights/TMVAMulticlass_MLP.weights.xml
<HEADER> MLP                      : Building Network. 
                         : Initializing weights
                         : Booked classifier "MLP" of type: "MLP"


Book output histograms

In [9]:
UInt_t nbin = 100;
TH1F *histMLP_signal(0), *histBDTG_signal(0), *histFDAGA_signal(0), *histPDEFoam_signal(0), *histDNNCPU_signal(0);
if (Use["MLP"])
   histMLP_signal    = new TH1F( "MVA_MLP_signal",    "MVA_MLP_signal",    nbin, 0., 1.1 );
if (Use["BDTG"])
   histBDTG_signal  = new TH1F( "MVA_BDTG_signal",   "MVA_BDTG_signal",   nbin, 0., 1.1 );
if (Use["DNN_CPU"]) histDNNCPU_signal = new TH1F("MVA_DNNCPU_signal", "MVA_DNNCPU_signal", nbin, 0., 1.1);
if (Use["FDA_GA"])
   histFDAGA_signal = new TH1F( "MVA_FDA_GA_signal", "MVA_FDA_GA_signal", nbin, 0., 1.1 );
if (Use["PDEFoam"])
   histPDEFoam_signal = new TH1F( "MVA_PDEFoam_signal", "MVA_PDEFoam_signal", nbin, 0., 1.1 );


TFile *input(0);
TString fname = "./tmva_example_multiple_background.root";
if (!gSystem->AccessPathName( fname )) {
   input = TFile::Open( fname ); // check if file in local directory exists
}
if (!input) {
   std::cout << "ERROR: could not open data file, please generate example data first!" << std::endl;
   exit(1);
}
std::cout << "--- TMVAMulticlassApp : Using input file: " << input->GetName() << std::endl;

--- TMVAMulticlassApp : Using input file: ./tmva_example_multiple_background.root


Prepare the tree
 - here the variable names have to corresponds to your tree
 - you can use the same variables as above which is slightly faster,
   but of course you can use different ones and copy the values inside the event loop

In [10]:
TTree* theTree = (TTree*)input->Get("TreeS");
std::cout << "--- Select signal sample" << std::endl;
theTree->SetBranchAddress( "var1", &var1 );
theTree->SetBranchAddress( "var2", &var2 );
theTree->SetBranchAddress( "var3", &var3 );
theTree->SetBranchAddress( "var4", &var4 );

std::cout << "--- Processing: " << theTree->GetEntries() << " events" << std::endl;
TStopwatch sw;
sw.Start();

for (Long64_t ievt=0; ievt<theTree->GetEntries();ievt++) {
   if (ievt%1000 == 0){
      std::cout << "--- ... Processing event: " << ievt << std::endl;
   }
   theTree->GetEntry(ievt);

   if (Use["MLP"])
      histMLP_signal->Fill((reader->EvaluateMulticlass( "MLP method" ))[0]);
   if (Use["BDTG"])
      histBDTG_signal->Fill((reader->EvaluateMulticlass( "BDTG method" ))[0]);
   if (Use["DNN_CPU"]) histDNNCPU_signal->Fill((reader->EvaluateMulticlass("DNN_CPU method"))[0]);
   if (Use["FDA_GA"])
      histFDAGA_signal->Fill((reader->EvaluateMulticlass( "FDA_GA method" ))[0]);
   if (Use["PDEFoam"])
      histPDEFoam_signal->Fill((reader->EvaluateMulticlass( "PDEFoam method" ))[0]);

}

--- Select signal sample
--- Processing: 2000 events
--- ... Processing event: 0
--- ... Processing event: 1000


Get elapsed time

In [11]:
sw.Stop();
std::cout << "--- End of event loop: "; sw.Print();

TFile *target  = new TFile( "TMVAMulticlassApp.root","RECREATE" );
if (Use["MLP"])
   histMLP_signal->Write();
if (Use["BDTG"])
   histBDTG_signal->Write();
if (Use["DNN_CPU"]) histDNNCPU_signal->Write();
if (Use["FDA_GA"])
   histFDAGA_signal->Write();
if (Use["PDEFoam"])
   histPDEFoam_signal->Write();

target->Close();
std::cout << "--- Created root file: \"TMVMulticlassApp.root\" containing the MVA output histograms" << std::endl;

delete reader;

std::cout << "==> TMVAMulticlassApp is done!" << std::endl << std::endl;

--- End of event loop: Real time 0:00:00, CP time 0.640
--- Created root file: "TMVMulticlassApp.root" containing the MVA output histograms
==> TMVAMulticlassApp is done!

